In [2]:
# Define helper functions 

import itertools
from itertools import chain
import random

def defvars(label, n):
    return list(var(label + '_%d' % i) for i in range(n))

def defpoly_from_basis(label, basis):
    coeffs = defvars(label,len(basis))
    poly = sum(c*x for c,x in zip(coeffs,basis))
    return (coeffs,poly)

def defpoly(label, d):
    return defpoly_from_basis(label, list(x**i for i in range(d+1)))

def concat(lists):
    return list(itertools.chain.from_iterable(lists))

# For given d, let P = prod_{j=0}^{d-1} (X-j), then this function
# returns the coefficient of P corresponding to X^i for given i
def stirling(d,i):
    if i == 0:
        return 0
    stirlingR = PolynomialRing(SR,'stir')
    p = prod(stirlingR.0-j for j in range(d))
    print(p)
    print(p.coefficients(stirlingR.0)[i-1])
    
def v_coeff(i,j,x):
    return binomial(i,j)*(x**(i-j))

#print(v_coeff(1,1,var('bla')))
#print(concat([[5,6],[6],[2,1,1]]))

def subs_vec(vec,subsmap):
    return vector([e.subs(subsmap) for e in vec])

def subs_mat(mat,subsmap):
    return Matrix([[e.subs(subsmap) for e in row] for row in mat])

# Vertically stacks two vectors
def stack(vec1,vec2):
    return vector(list(vec1) + list(vec2))

def print_coeff(eq,mon):
    show(mon)
    if mon == 1:
        print(eq.constant_coefficient())
    else:
        print(eq.monomial_coefficient(mon))
    print("--------------------------")

In [4]:
# Define the language. 
# Define the matrix parameters n,m. 
# Define the matrix M, define vector tau.

#n = 3              # number of equations
#nx = 3             # number of instance elements
#m = 2              # number of witness elements

#hs = [var('H0')]

# For uBlu
d = 3
n = 3*d+4
nx = 3 + 2*d
m = 2*d+8

hs = [var('H0'),var('H1')] + list(var('W%d' % (i+1)) for i in range(d)) # Some hash-to-curve elements
nh = len(hs)

# A function that generates a random language.
def generate_random_language():
    mat = [[0 for j in range(m)] for i in range(n)]
    for i in range(n):
        for j in range(m):
            choice = random.random()
            if choice < 0.6:
                mat[i][j] = lambda inst: 0
            elif choice < 0.95:
                mat[i][j] = lambda inst: 1
            else:
                ix = random.randint(0, nx-1)
                mat[i][j] = lambda inst: inst[ix]

    vec = [0 for i in range(n-nx)]
    for i in range(n-nx):
        # Each element is randomly either 0, 1, or inst[i]
        choice = random.random()
        if choice < 0.33:
            vec[i] = lambda inst: 0
        elif choice < 0.66:
            vec[i] = lambda inst: 1
        else:
            ix = random.randint(0, nx-1)
            vec[i] = lambda inst: inst[ix]
    
    def random_M(inst):
        mat_instantiated = [[mat[i][j](inst) for j in range(m)] for i in range(n)]
        return Matrix(mat_instantiated)

    def random_theta(inst):
        vec_instantiated = [vec[i](inst) for i in range(n-nx)]
        return vector(vec_instantiated)
        
    return random_M, random_theta


# Generate matrices automatically
# M, theta = generate_random_language()

# Custom matrix definition
def M(inst):

    mat = [[0 for j in range(m)] for i in range(n)]
    mat[0][0] = 1
    mat[0][1] = hs[0]
    mat[1][2] = 1
    mat[1][3] = hs[0]
    mat[2][4] = 1
    mat[2][5] = hs[0]
    mat[3][9] = 1   # A_1 = G^{r_1}
    mat[4][6] = 1
    mat[4][9] = hs[1] 
    mat[4][4] = hs[2]
    for i in range(d-1):
        mat[5+2*i][10+i] = 1
        mat[5+2*i+1][6] = inst[5+2*i-1]
        mat[5+2*i+1][9+d+i] = -hs[1]
        mat[5+2*i+1][10+i] = hs[1]
        mat[5+2*i+1][7] = -hs[2+i]
        mat[5+2*i+1][4] = hs[3+i]

    mat[3+2*d][2] = 1    
    mat[3+2*d][0] = -1    
    mat[3+2*d][6] = -1
    mat[3+2*d+1][6] = inst[2]
    mat[3+2*d+1][7] = -1
    mat[3+2*d+1][8] = -hs[0]
    
    for i in range(d-1):
        mat[3+2*d+2+i][6] = inst[3+2*i]
        mat[3+2*d+2+i][9+d+i] = -1
        
    return Matrix(mat)

# Define vector theta
def theta(inst):    
     vec = [1 for i in range(n-nx)] # uBlu theta is just 1s

     return vector(vec)
    

xs = [var('x_%d' % i) for i in range(nx)]
ws = [var('w_%d' % i) for i in range(m)]
x = vector(xs)
w = vector(ws)

# z variables are akin to w, but used for modelling "any witness" for blinding compatibility
zs = [var('z_%d' % i) for i in range(m)]
z = vector(zs)

assert(n == M(xs).nrows())
assert(m == M(xs).ncols())

print(M(xs))

print(theta(xs))

# 
def find_coeff_of_var(expr, var):
    return expr.coefficient(var, 1)

def find_constant_coeff(expr):
    return expr.subs({v : 0 for v in expr.variables()})

# Matrix m as a tensor
m_tens = [[[find_coeff_of_var(M(xs)[i][j], xs[t]) for t in range(nx)] for j in range(m)] for i in range(n)]
m_tens_h = [[[find_coeff_of_var(M(xs)[i][j], hs[t]) for t in range(len(hs))] for j in range(m)] for i in range(n)]
m_tens_const = [[find_constant_coeff(M(xs)[i][j]) for j in range(m)] for i in range(n)]

for t in range(nx):
    print("main inst t: ", t)
    print(Matrix([[m_tens[i][j][t] for j in range(m)] for i in range(n)]))

for t in range(nh):
    print("h t: ", t)
    print(Matrix([[m_tens_h[i][j][t] for j in range(m)] for i in range(n)]))

print("const: ")
print(Matrix(m_tens_const))

[  1  H0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   1  H0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   1  H0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   1   0   0   0   0]
[  0   0   0   0  W1   0   1   0   0  H1   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   1   0   0   0]
[  0   0   0   0  W2   0 x_4 -W1   0   0  H1   0 -H1   0]
[  0   0   0   0   0   0   0   0   0   0   0   1   0   0]
[  0   0   0   0  W3   0 x_6 -W2   0   0   0  H1   0 -H1]
[ -1   0   1   0   0   0  -1   0   0   0   0   0   0   0]
[  0   0   0   0   0   0 x_2  -1 -H0   0   0   0   0   0]
[  0   0   0   0   0   0 x_3   0   0   0   0   0  -1   0]
[  0   0   0   0   0   0 x_5   0   0   0   0   0   0  -1]
(1, 1, 1, 1)
main inst t:  0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 

In [ ]:
# A comment-only block collecting some manual languages.

# def M(inst):
#     mat = [[0 for j in range(m)] for i in range(n)]
#     # mat[0][0] = 1         # x1 = G^w1
#     # mat[1][1] = hs[0]     # x2 = H^w2
#     # mat[2][1] = inst[0]   # x3 = x1^w2 = G^{w1 w2}
    
#     # mat[3][2] = 1         # x3 = G^{w3}
    
#     # mat[4][0] = inst[1]   # x3^{-1} = x2^w1 (H^-1 G^-1)^w3
#     # mat[4][2] = (-1 - hs[0])

#     # Powers of 1 witness plus 2nd witness mixed in, no blinders
#     mat[0][0] = 1         # x1 = G^w1 
#     mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
#     mat[2][1] = inst[1]   # x3 = x2^w2 = G^{w1^2 w2}
        
#     return Matrix(mat)

# def theta(inst):
#     vec = [0 for i in range(n-nx)]

#     # vec[0] = inst[2]
#     # vec[1] = -inst[2]

#     return vector(vec)

############ (?) BC
### The following language has 4 transformations, and transformation # 2 is not generically blinding-compatible: 
### It is only blinding compatible if r9: r46 + r47, where r47 is a new Ta associated variable, and r46 is Tx variable.
### @Volhovm: this is OK up to variable replacement: r46+r47 -> alpha, r47 -> beta, then r46 = alpha - beta.

# def M(inst):
#     mat = [[0 for j in range(m)] for i in range(n)]
# 
#     mat[0][0] = 1         # x1 = G^w1
#     mat[1][1] = inst[0]   # x2 = x1^w1 = G^{w1 w2}
#     mat[2][2] = 1         # x1 * x2 = G^w3
#         
#     return Matrix(mat)
# 
# def theta(inst):
#     vec = [0 for i in range(n-nx)]
# 
#     vec[0] = inst[0] + inst[1]         # x1 * x3
# 
#     return vector(vec)
# 
#  solution # 1:
# {Tx2_0: r8, Tx2_1: r11*r8, Tx1_0_0: r9, Tx1_0_1: r10, Tx1_1_0: r11*r9, Tx1_1_1: r10*r11, Tw2_0: r8, Tw2_1: r11, Tw2_2: (r11 + 1)*r8, Tw1_0_0: -r10 + r9, Tw1_0_1: 0, Tw1_0_2: r10, Tw1_1_0: 0, Tw1_1_1: 0, Tw1_1_2: 0, Tw1_2_0: -r10*(r11 + 1) + (r11 + 1)*r9, Tw1_2_1: 0, Tw1_2_2: r10*(r11 + 1)}
#  Tw1:
# [                    -r10 + r9                             0                           r10]
# [                            0                             0                             0]
# [-r10*(r11 + 1) + (r11 + 1)*r9                             0                 r10*(r11 + 1)]
#  Tw2:
# (r8, r11, (r11 + 1)*r8)
#  Tx1:
# [     r9     r10]
# [ r11*r9 r10*r11]
#  Tx2:
# (r8, r11*r8)



######### (?) BC
# This one has a single transformation, but two distinct BC variants of that, not one unified BC variant. Why?
#    mat[0][0] = 1         # x1 = G^w1
#    mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
#    mat[2][1] = 1         # x2 = G^w2       => w2 = w1^2
#    mat[3][2] = 1         # x2 = G^w3       => w3 = w1^2

# BC (attempts)
    # Powers of 1 witness plus 2nd witness mixed in, no blinders
    #mat[0][0] = 1         # x1 = G^w1 
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
    #mat[2][1] = inst[1]   # x3 = x2^w2 = G^{w1^2 w2}

    # Powers of 1 witness
    #mat[0][0] = 1         # x1 = G^w1
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
    #mat[2][0] = inst[1]   # x3 = x2^w1 = G^{w1^3}

    # Powers of 1 witness plus some blinders?
    #mat[0][0] = 1         # x1 = G^w1 H^w3
    #mat[0][2] = hs[0]
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2} H^{w3 w1}
    #mat[2][1] = inst[1]   # x3 = x2^w2 H^w4 = G^{w1^2 w2} H^{w3 w1 w2 + w4}
    #mat[2][3] = hs[0]


# Tx1 * x + Tx2
# Tx1 * (x || H) + Tx2


# transformation ???
#
# x1' = x1 * G^alpha
# x2' = x2 * x1^{2 alpha} * G^{alpha^2}
# x3' = x3 * 
#  (w1 + alpha)^2 w2 = w1 w2 + 2 alpha w1 w2 + alpha^2 w2

In [5]:
# Define generic transformation matrices that we can solve for later

Tx1s = [[var('Tx1_%d_%d' % (i, j)) for j in range(nx)] for i in range(nx)]
Tx2s = [var('Tx2_%d' % i) for i in range(nx)]
Tw1s = [[var('Tw1_%d_%d' % (i,j)) for j in range(m)] for i in range(m)]
Tw2s = [var('Tw2_%d' % i) for i in range(m)]


#T_reduce_map = {var('U_α'): 0, var('w_α'): 0}
T_reduce_map = {}
Tx1 = Matrix(subs_mat(Tx1s,T_reduce_map))
Tx2 = vector(subs_vec(Tx2s,T_reduce_map))
Tw1 = Matrix(subs_mat(Tw1s,T_reduce_map))
Tw2 = vector(subs_vec(Tw2s,T_reduce_map))
 

print(Tw1)
print(Tw2)
print("--------------------")
print(Tx1)
print(Tx2)
print("--------------------")
#print(us)

# Backups
Tx1s_orig = Tx1s
Tx2s_orig = Tx2s
Tw1s_orig = Tw1s
Tw2s_orig = Tw2s

Tx1_orig = Tx1
Tx2_orig = Tx2
Tw1_orig = Tw1
Tw2_orig = Tw2

ts_map = {}

[  Tw1_0_0   Tw1_0_1   Tw1_0_2   Tw1_0_3   Tw1_0_4   Tw1_0_5   Tw1_0_6   Tw1_0_7   Tw1_0_8   Tw1_0_9  Tw1_0_10  Tw1_0_11  Tw1_0_12  Tw1_0_13]
[  Tw1_1_0   Tw1_1_1   Tw1_1_2   Tw1_1_3   Tw1_1_4   Tw1_1_5   Tw1_1_6   Tw1_1_7   Tw1_1_8   Tw1_1_9  Tw1_1_10  Tw1_1_11  Tw1_1_12  Tw1_1_13]
[  Tw1_2_0   Tw1_2_1   Tw1_2_2   Tw1_2_3   Tw1_2_4   Tw1_2_5   Tw1_2_6   Tw1_2_7   Tw1_2_8   Tw1_2_9  Tw1_2_10  Tw1_2_11  Tw1_2_12  Tw1_2_13]
[  Tw1_3_0   Tw1_3_1   Tw1_3_2   Tw1_3_3   Tw1_3_4   Tw1_3_5   Tw1_3_6   Tw1_3_7   Tw1_3_8   Tw1_3_9  Tw1_3_10  Tw1_3_11  Tw1_3_12  Tw1_3_13]
[  Tw1_4_0   Tw1_4_1   Tw1_4_2   Tw1_4_3   Tw1_4_4   Tw1_4_5   Tw1_4_6   Tw1_4_7   Tw1_4_8   Tw1_4_9  Tw1_4_10  Tw1_4_11  Tw1_4_12  Tw1_4_13]
[  Tw1_5_0   Tw1_5_1   Tw1_5_2   Tw1_5_3   Tw1_5_4   Tw1_5_5   Tw1_5_6   Tw1_5_7   Tw1_5_8   Tw1_5_9  Tw1_5_10  Tw1_5_11  Tw1_5_12  Tw1_5_13]
[  Tw1_6_0   Tw1_6_1   Tw1_6_2   Tw1_6_3   Tw1_6_4   Tw1_6_5   Tw1_6_6   Tw1_6_7   Tw1_6_8   Tw1_6_9  Tw1_6_10  Tw1_6_11  Tw1_6_12  Tw1_6_13]
[  Tw1

In [6]:
# Define the (concrete) ublu transformation matrix 

Tx1s = [[0 for j in range(n)] for i in range(n)]
Tx2s = [0 for i in range(n)]
Tx3s = [[0 for j in range(nh)] for i in range(n)]
Tw1s = [[0 for j in range(m)] for i in range(m)]
Tw2s = [0 for i in range(m)]

for i in range(n):
    Tx1s[i][i] = 1
    
for i in range(m):
    Tw1s[i][i] = 1
 

us = [var('U_x'),var('U_rx'),var('U_α'),var('U_rα')] + \
     [var('U_r%d' % (i+1)) for i in range(d)] 


Tw2s[2] = var('U_x') # x + U_x
Tw2s[3] = var('U_rx') # rx + U_rx
Tw2s[4] = var('U_α') # α + U_α
Tw2s[5] = var('U_rα') # rα + U_rα
Tw2s[6] = var('U_x') # (x-t) + U_x
Tw1s[7][4] = var('U_x') # α*(x-t) + α*U_x
Tw1s[7][6] = var('U_α') #         + (x-t)*U_α
Tw2s[7] = var('U_x')*var('U_α') # + U_x*U_α
Tw1s[8][5] = var('U_x') # rα*(x-t) + rα*U_x
Tw1s[8][6] = var('U_rα') #         + (x-t)*U_rα
Tw2s[8] = var('U_x')*var('U_rα') # + U_x*U_α

for i in range(d):
    for j in range(i+1):
        Tw1s[9+i][9+j] = v_coeff(i+1,j+1,var('U_x'))
    Tw2s[9+i] = us[4+i]

for i in range(d-1):
    for j in range(i+1):
        Tw1s[9+d+i][9+d+j] = v_coeff(i+1,j+1,var('U_x'))
        Tw1s[9+d+i][9+j] = var('U_x') * v_coeff(i+1,j+1,var('U_x'))
    Tw1s[9+d+i][6] = us[4+i] 
    Tw2s[9+d+i] = us[4+i] * var('U_x')

# Xcal * G^{U_x} * H0^{U_rx}
Tx2s[1] = var('U_x') 
Tx3s[1][0] = var('U_rx') 
# Acal * G^{U_α} * H0^{U_rα}
Tx2s[2] = var('U_α')
Tx3s[2][0] = var('U_rα') 

for i in range(d):
    for j in range(i+1):
        Tx1s[3+2*i][3+2*j] = v_coeff(i+1,j+1,var('U_x'))
        Tx1s[4+2*i][4+2*j] = v_coeff(i+1,j+1,var('U_x'))
    Tx2s[3+2*i] = us[4+i]
    Tx2s[4+2*i] = var('U_x')**(i+1) 
    
    Tx3s[4+2*i][2+i] = ws[4] + var('U_α')
    Tx3s[4+2*i][1] = us[4+i]
    for j in range(i+1):
        Tx3s[4+2*i][2+j] = -ws[4]*v_coeff(i+1,j+1,var('U_x'))



#T_reduce_map = {var('U_α'): 0, var('w_α'): 0}
T_reduce_map = {}
Tx1 = Matrix(subs_mat(Tx1s,T_reduce_map))
Tx2 = vector(subs_vec(Tx2s,T_reduce_map))
Tx3 = Matrix(subs_mat(Tx3s,T_reduce_map))
Tw1 = Matrix(subs_mat(Tw1s,T_reduce_map))
Tw2 = vector(subs_vec(Tw2s,T_reduce_map))
 

print(Tw1)
print(Tw2)
print("--------------------")
print(Tx1)
print(Tx2)
print(Tx3)
print("--------------------")
print(us)

[      1       0       0       0       0       0       0       0       0       0       0       0       0       0]
[      0       1       0       0       0       0       0       0       0       0       0       0       0       0]
[      0       0       1       0       0       0       0       0       0       0       0       0       0       0]
[      0       0       0       1       0       0       0       0       0       0       0       0       0       0]
[      0       0       0       0       1       0       0       0       0       0       0       0       0       0]
[      0       0       0       0       0       1       0       0       0       0       0       0       0       0]
[      0       0       0       0       0       0       1       0       0       0       0       0       0       0]
[      0       0       0       0     U_x       0     U_α       1       0       0       0       0       0       0]
[      0       0       0       0       0     U_x    U_rα       0       1       0       0

In [7]:
# Define our main equations: the verification ("basic") equation, and the update equation.

# Optional: perform a substitution mapping for instance/witness variables
#tsmod = {t_α: 0, t_rα: 0}
tsmod = {}
x2 = vector([x[i] * (1 if x[i].subs(ts_map).subs(tsmod).full_simplify() != 0 else 0) for i in range(nx)])
w2 = vector([w[i] * (1 if w[i].subs(ts_map).subs(tsmod).full_simplify() != 0 else 0) for i in range(m)])
x2w2_map = {ws[i]:w2[i] for i in range(m)} | {xs[i]:x2[i] for i in range(len(xs))}
print(x2w2_map)
print("-------")

Tx1 = subs_mat(Tx1,x2w2_map)
Tx2 = subs_vec(list(Tx2),x2w2_map)
# Tx1_top = Tx1[0:nx] # excluding theta
Tx1 = Tx1.submatrix(0,0,nx,nx)
Tx2 = Tx2[0:nx] # excluding theta
Tw1 = subs_mat(Tw1,x2w2_map)
Tw2 = subs_vec(list(Tw2),x2w2_map)

# Our main equation
eq_basic = stack(x2,theta(x2)) - M(x2) * w2

print("Main equation, by instance vector component 0..n:")
for i in range(n):
    print(" ", i, ": ", eq_basic[i].full_simplify())
print("-------")

# Update equation
#eq_u = Tx1 * (M(x2) * w2) + Tx2 - (M(Tx1 * x2 + Tx2) * (Tw1 * w2 + Tw2))
x_upd = Tx1 * x + Tx2
#x_upd_2 = Tx1_top * (M(x) * w) + Tx2_top
w_upd = Tw1 * w + Tw2

eq_u = stack(x_upd, theta(x_upd)) - M(Tx1 * x + Tx2) * w_upd

#eq_u = stack(Tx1_top * (M(x) * w) + Tx2_top, theta(Tx1_top * (M(x) * w) + Tx2_top)) - (M(Tx1 * stack(x,theta(x)) + Tx2) * (Tw1 * w + Tw2))
print("Update equation, by instance vector component 0..n:")
for (e,i) in zip(eq_u,range(n)):
    print(" ", i, ": ", e.full_simplify())

{w_0: w_0, w_1: w_1, w_2: w_2, w_3: w_3, w_4: w_4, w_5: w_5, w_6: w_6, w_7: w_7, w_8: w_8, w_9: w_9, w_10: w_10, w_11: w_11, w_12: w_12, w_13: w_13, x_0: x_0, x_1: x_1, x_2: x_2, x_3: x_3, x_4: x_4, x_5: x_5, x_6: x_6, x_7: x_7, x_8: x_8}
-------
Main equation, by instance vector component 0..n:
  0 :  -H0*w_1 - w_0 + x_0
  1 :  -H0*w_3 - w_2 + x_1
  2 :  -H0*w_5 - w_4 + x_2
  3 :  -w_9 + x_3
  4 :  -W1*w_4 - H1*w_9 - w_6 + x_4
  5 :  -w_10 + x_5
  6 :  -H1*w_10 + H1*w_12 - W2*w_4 + W1*w_7 - w_6*x_4 + x_6
  7 :  -w_11 + x_7
  8 :  -H1*w_11 + H1*w_13 - W3*w_4 + W2*w_7 - w_6*x_6 + x_8
  9 :  w_0 - w_2 + w_6 + 1
  10 :  H0*w_8 - w_6*x_2 + w_7 + 1
  11 :  -w_6*x_3 + w_12 + 1
  12 :  -w_6*x_5 + w_13 + 1
-------
Update equation, by instance vector component 0..n:
  0 :  -H0*w_1 - w_0 + x_0
  1 :  -H0*U_rx - H0*w_3 - w_2 + x_1
  2 :  -H0*U_rα - H0*w_5 - w_4 + x_2
  3 :  -w_9 + x_3
  4 :  -H1*U_r1 - U_α*W1 - W1*w_4 - H1*w_9 - w_6 + x_4
  5 :  -2*U_x*w_9 + 2*U_x*x_3 - w_10 + x_5
  6 :  H1*U_r1*

In [8]:
# Solve for dependencies between witnesses. 
# 
# It seems only reasonable to AGM-like solve over the independent basis 
# of trapdoors, and if w_i are dependent, we should first express them in terms of a basis.
#
# E.g. w1 = r1, w2 = r1^2, w3 = r1^3, and same for x.

x2_flat = [elem for elem in x2]
w2_flat = [elem for elem in w2]
hs_flat = [elem for elem in hs]
print(x2_flat + w2_flat + hs_flat)

#eq_basic_updated = [sym_to_poly(eq) for eq in eq_basic]
#print(eq_basic_updated)
sols = solve(list(eq_basic), w2_flat + x2_flat + hs_flat, solution_dict=True)
print(sols)

# some solutions will be partial and assume e.g. hs[0] = 0, which is unrealistic.
# We only want to keep solutions where hs[i] is free variable
# we only keep these solutions, and remove hs keys from it, replacing these free variables
# by hs[i] variables directly
def filter_sols_with_hs_free(sols):
    retset = []
    for sol in sols:
        if any([len(sol[hs_i].variables()) != 1 for hs_i in hs]):
            continue

        if len(set(list([sol[hs_i].variables() for hs_i in hs]))) != len(hs):
            continue

        modsol = {k: v.subs({sol[hs_i]: hs_i for hs_i in hs}) for k,v in sol.items()}
        for hs_i in hs:
            modsol.pop(hs_i)
        
        retset.append(modsol)
    
    return retset

sols = filter_sols_with_hs_free(sols)
print("filtered len: ", len(sols))
print("filtered: ", sols)

chosen_sol = sols[0]

# We need solutions that leave hs as fully free variables.

def get_basis_from_sols(sol):
    # Assuming sols[0] is your solution dictionary
    params = set()  # find all parameters (like r4)
    for value in sol.values():
        params.update(value.variables())

    return list(params)

eq_basic_params = [x for x in get_basis_from_sols(chosen_sol) if x not in hs]
print("eq_basic_params: ", eq_basic_params)

eq_u_via_params = [eq.subs(chosen_sol).full_simplify() for eq in eq_u]
print("eq_u_via_params: ", eq_u_via_params)

x_via_eq_basic_params = vector([elem.subs(chosen_sol) for elem in x2])
print(x_via_eq_basic_params)

[x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, w_0, w_1, w_2, w_3, w_4, w_5, w_6, w_7, w_8, w_9, w_10, w_11, w_12, w_13, H0, H1, W1, W2, W3]
[{x_0: r1, x_1: r2, x_2: r3, x_3: r4, x_4: -r11*r13*r9 + r13*r3 + r12*r4 + r11*r7 - r11*r8 - r1 + r2 - 1, x_5: r5, x_6: r11^2*r7^2 + r11^2*r8^2 + r10*r11*r13 + (r11*r13*r9 - 2*r11*r7 + 2*r11*r8 + 2)*r1 + r1^2 - (r11*r13*r9 - 2*r11*r7 + 2*r11*r8 + 2*r1 + 2)*r2 + r2^2 + r14*r3 + r12*r5 - 2*r11*r7 - 2*(r11^2*r7 - r11)*r8 - (r11^2*r13*r7 - r11^2*r13*r8 - r11*r13 + r11*r14)*r9 + r12 + r13 + 1, x_7: r6, x_8: r11^3*r7^3 - r11^3*r8^3 - 3*r11^2*r7^2 - (r11*r13*r9 - 3*r11*r7 + 3*r11*r8 + 3)*r1^2 - r1^3 - (r11*r13*r9 - 3*r11*r7 + 3*r11*r8 + 3*r1 + 3)*r2^2 + r2^3 + 3*(r11^3*r7 - r11^2)*r8^2 - (3*r11^2*r7^2 + 3*r11^2*r8^2 + r10*r11*r13 - 6*r11*r7 - 6*(r11^2*r7 - r11)*r8 - (2*r11^2*r13*r7 - 2*r11^2*r13*r8 - 2*r11*r13 + r11*r14)*r9 + r12 + r13 + 3)*r1 + (r11^2*r13*r7 - r11^2*r13*r8 - r11*r13 + r11*r14)*r10 + (3*r11^2*r7^2 + 3*r11^2*r8^2 + r10*r11*r13 + 2*(r11*r13

In [9]:
print(Tw2)

(0, 0, U_x, U_rx, U_α, U_rα, U_x, U_x*U_α, U_rα*U_x, U_r1, U_r2, U_r3, U_r1*U_x, U_r2*U_x)


In [10]:
# Helper functions for solving equations AGM-style
# - Symbolic form is how equations are defined without a ring
# - "poly" or ring form is polynomials over a ring. only in this form we can extract monomials.

# Takes a ring and a list of variables, and re-assigns these variables to a ring.
# Returns:
#   - to_sym_map is a map that maps a ring variable to the original symbolic base variable
#   - ret is a list of mapped variables in a ring
def reassign_vars(R,varlists):
    ret = []
    totsum = 0
    to_sym_map = {} 
    for base_vars in varlists:
        ring_vars = [R.gen(i) for i in range(totsum,totsum+len(base_vars))]
        ret.append(ring_vars)
        to_sym_map = to_sym_map | {ring_vars[i]:base_vars[i] for i in range(len(base_vars))} 
        totsum += len(base_vars)
    return (to_sym_map,ret)

# Ringvars are the "Trapdoors" over which we want to extract AGM sub-equations
ringvars = [hs, xs, ws, zs, eq_basic_params]
R = PolynomialRing(SR, concat(ringvars))
#R.inject_variables()
(to_sym_map,[poly_hs,poly_xs,poly_ws,poly_zs, poly_eq_basic_params]) = reassign_vars(R,ringvars)

print(ringvars)

# Convert a polynomial from ring form to the symbolic form
def poly_to_sym(poly):
    print(poly.subs(to_sym_map))
    return poly.subs(to_sym_map)

# Convert a polynomial from symbolic form to ring form
def sym_to_poly(poly):
    from sage.symbolic.expression_conversions import polynomial
    return polynomial(poly, ring=R)

[[H0, H1, W1, W2, W3], [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8], [w_0, w_1, w_2, w_3, w_4, w_5, w_6, w_7, w_8, w_9, w_10, w_11, w_12, w_13], [z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7, z_8, z_9, z_10, z_11, z_12, z_13], [r10, r4, r2, r9, r7, r6, r5, r1, r8, r3]]


In [56]:
# Find basic wit/inst transformations.
#
# AGM-(like-)solve for witness/instance transformations (Tx, Tw) but NOT Ta
#
# *TODO* rn it's not AGM because adversary can't see the instance
# Assume the most generic form of Tw
# Model Tx as an algebraic sum of instance elements and fixed elements like H
# Take update equations, line by line, and extract sub-equations by monomial coefficients in trapdoor vars
# Trapdoor vars (secrets) are ... witness elements I think, plus DLOGs of hash-to-curve els

Tx_vars_flat = Tx2s + [x for sublist in Tx1s for x in sublist]
print(Tx_vars_flat)
print("-------")

Tw_vars_flat = Tw2s + [x for sublist in Tw1s for x in sublist]


# Transform equation to symbolic form
#eq_u_updated = [sym_to_poly(eq.subs(basic_sol)) for eq in eq_u]
eq_u_updated = [sym_to_poly(eq) for eq in eq_u_via_params]
#eq_u_updated = [sym_to_poly(eq.subs({k:basic_sol[k] for k in xs})) for eq in eq_u]
print("updated eqs: ", eq_u_updated)
print("-------")

all_monomials = list(dict.fromkeys([mon for eq in eq_u_updated for mon in eq.monomials()]))
print("all monomials: ", all_monomials)
print("-------")
    
coeff_eqs = [eq.monomial_coefficient(mon) for eq in eq_u_updated for mon in eq.monomials() ]
print("coeff_eqs: ", coeff_eqs)
for (e,i) in zip(coeff_eqs,range(0,len(coeff_eqs))):
    print(" ", i, ": ", e)
print("-------")

sols = solve(coeff_eqs,Tx_vars_flat + Tw_vars_flat, solution_dict=True)


print("------------ ", len(sols)," solutions")


def is_special_case_of(sol1, sol2):
    """Check if sol1 is a special case of sol2"""
    # Create a system of equations comparing the solutions
    equations = []
    variables = set()
    
    for var in sol2:
        eq = sol1[var] == sol2[var]
        equations.append(eq)
        variables.update(sol2[var].variables())

    # Try to solve for the variables in sol2 that would make sol1 = sol2
    result = solve(equations, list(variables))
    return len(result) > 0  # If there's a solution, sol1 is a special case

def filter_generic_solutions(solutions):
    filtered = []
    for i, sol1 in enumerate(solutions):
        is_most_generic = True
        for j, sol2 in enumerate(solutions):
            if i != j:
                # Try substituting sol1 into sol2 to check if sol1 is more specific
                try:
                    if is_special_case_of(sol1,sol2):  # if substitution works, sol1 is less generic
                        is_most_generic = False
                        break
                except:
                    continue
        if is_most_generic:
            filtered.append(sol1)
    return filtered

sols = filter_generic_solutions(sols)

print("------------ ", len(sols)," FILTERED (generic) solutions")
for sol_i,sol in enumerate(sols):
    print("============\n solution # %d:" % sol_i)
    print(sol)

    print(" Tw1:")
    print(Tw1.subs(sol))
    print(" Tw2:")
    print(Tw2.subs(sol))


    print(" Tx1:")
    print(Tx1.subs(sol))
    print(" Tx2:")
    print(Tx2.subs(sol))

    #TODO Test solutions for validity


[Tx2_0, Tx2_1, Tx2_2, Tx1_0_0, Tx1_0_1, Tx1_0_2, Tx1_1_0, Tx1_1_1, Tx1_1_2, Tx1_2_0, Tx1_2_1, Tx1_2_2]
-------
updated eqs:  [Tx1_0_2*r84*r83^2 + Tx1_0_1*r83^2 + (-Tw1_0_1)*r84 + (-Tw1_0_0 + Tx1_0_0)*r83 - Tw2_0 + Tx2_0, (-Tw1_0_1*Tx1_0_2)*r84^2*r83^2 + (-Tw1_0_0*Tx1_0_2)*r84*r83^3 + (-Tw1_0_1*Tx1_0_1 - Tw2_0*Tx1_0_2 + Tx1_1_2)*r84*r83^2 + (-Tw1_0_0*Tx1_0_1)*r83^3 + (-Tw1_0_1*Tx1_0_0)*r84*r83 + (-Tw1_0_0*Tx1_0_0 - Tw2_0*Tx1_0_1 + Tx1_1_1)*r83^2 + (-Tw1_0_1*Tx2_0)*r84 + (-Tw2_0*Tx1_0_0 - Tw1_0_0*Tx2_0 + Tx1_1_0)*r83 - Tw2_0*Tx2_0 + Tx2_1, (-Tw1_1_1*Tx1_1_2)*r84^2*r83^2 + (-Tw1_1_0*Tx1_1_2)*r84*r83^3 + (-Tw1_1_1*Tx1_1_1 - Tw2_1*Tx1_1_2 + Tx1_2_2)*r84*r83^2 + (-Tw1_1_0*Tx1_1_1)*r83^3 + (-Tw1_1_1*Tx1_1_0)*r84*r83 + (-Tw1_1_0*Tx1_1_0 - Tw2_1*Tx1_1_1 + Tx1_2_1)*r83^2 + (-Tw1_1_1*Tx2_1)*r84 + (-Tw2_1*Tx1_1_0 - Tw1_1_0*Tx2_1 + Tx1_2_0)*r83 - Tw2_1*Tx2_1 + Tx2_2]
-------
all monomials:  [r84*r83^2, r83^2, r84, r83, 1, r84^2*r83^2, r84*r83^3, r83^3, r84*r83]
-------
coeff_eqs:  [Tx1_0_2, Tx1_0_1

In [14]:
# Solve for TA matrix using the same AGM-like technique.
# Iterate over all the available transformations, and check if any of them are _not_ blinding compatible.

def solve_for_ta(chosen_sol):
    # Choose a _concrete_ update transformation matrix.
    
    print(chosen_sol)
    
    Tw1 = Tw1_orig.subs(chosen_sol)
    print(" Tw1:")
    print(Tw1)
    Tw2 = Tw2_orig.subs(chosen_sol)
    print(" Tw2:")
    print(Tw2)
    
    Tx1 = Tx1_orig.subs(chosen_sol)
    print(" Tx1:")
    print(Tx1)
    Tx2 = Tx2_orig.subs(chosen_sol)
    print(" Tx2:")
    print(Tx2)

    # Define blinding-compatible equation w.r.t. this new transformation matrix
    
    Ta1s = list(list(var('Ta1_%d_%d' % (i, j)) for j in range(2*n+1)) for i in range(n))
    Ta2s = list(var('Ta2_%d' % i) for i in range(n))
    
    Ta_vars_flat = Ta2s + [elem for sublist in Ta1s for elem in sublist]
    Tx_sol_vars = [elem for elem in get_basis_from_sols(chosen_sol)]
    
    Ta1 = Matrix(Ta1s)
    Ta2 = vector(Ta2s)
    
    # THIS EQUATION MUST BE OVER z and BASIS OF w, not over x.
    # Blinding compatible transformation equation
    x_alt = x_via_eq_basic_params
    eq_u_ch = Ta1 * vector(concat([M(x_alt) * z, x_alt, theta(x_alt), hs])) + Ta2 - (M(Tx1 * x_alt + Tx2) * (Tw1 * z + Tw2))
    
    print("Update equation, by instance vector component 0..n:")
    for (e,i) in zip(eq_u_ch,range(len(eq_u_ch))):
      print(" ", i, ": ", e)
    
    # Solving for Ta! Blinding compatibility.
    
    # Transform equation to symbolic form
    #eq_u_updated = [sym_to_poly(eq.subs(basic_sol)) for eq in eq_u]
    eq_u_ch_updated = [sym_to_poly(eq) for eq in eq_u_ch]
    #eq_u_updated = [sym_to_poly(eq.subs({k:basic_sol[k] for k in xs})) for eq in eq_u]
    print("updated eqs: ", eq_u_ch_updated)
    print("-------")
    
    all_monomials = list(dict.fromkeys([mon for eq in eq_u_ch_updated for mon in eq.monomials()]))
    print("all monomials: ", all_monomials)
    print("-------")
    
    coeff_eqs = [eq.monomial_coefficient(mon) for eq in eq_u_ch_updated for mon in eq.monomials() ]
    print("coeff_eqs: ", coeff_eqs)
    for (e,i) in zip(coeff_eqs,range(0,len(coeff_eqs))):
        print(" ", i, ": ", e)
    print("-------")

    sols = solve(coeff_eqs, Ta_vars_flat, solution_dict=True)

    if len(sols) == 0:
        print("          NO FULL solutions found ================================== ")
    
        partial_sols = solve(coeff_eqs, Ta_vars_flat+Tx_sol_vars, solution_dict=True)
    
        if len(partial_sols) == 0:
            for i in range(30):
                print("          NO SOLUTIONS FOUND ========================================")
            return
        else:
            print("          PARTIAL solution FOUND")
            sols = partial_sols

    print("------------ ", len(sols)," solutions")
    
    for sol_i,sol in enumerate(sols):
        print("============\n solution # %d:" % sol_i)
        print(sol)
        
        print(" Ta1:")
        print(Ta1.subs(sol))
        print(" Ta2:")
        print(Ta2.subs(sol))
        
        print(" Tw1:")
        print(Tw1.subs(sol))
        print(" Tw2:")
        print(Tw2.subs(sol))
        
        
        print(" Tx1:")
        print(Tx1.subs(sol))
        print(" Tx2:")
        print(Tx2.subs(sol))

    print("~~~~~~~~~~~~~~~~~~~")
    print("~~~~~~~~~~~~~~~~~~~\n\n\n")

for sol in sols:
    solve_for_ta(sol)

{x_0: r1, x_1: r2, x_2: r3, x_3: r4, x_4: -H0*W1*r9 + W1*r3 + H1*r4 + H0*r7 - H0*r8 - r1 + r2 - 1, x_5: r5, x_6: H0^2*r7^2 + H0^2*r8^2 + H0*W1*r10 + (H0*W1*r9 - 2*H0*r7 + 2*H0*r8 + 2)*r1 + r1^2 - (H0*W1*r9 - 2*H0*r7 + 2*H0*r8 + 2*r1 + 2)*r2 + r2^2 + W2*r3 + H1*r5 - 2*H0*r7 - 2*(H0^2*r7 - H0)*r8 - (H0^2*W1*r7 - H0^2*W1*r8 - H0*W1 + H0*W2)*r9 + H1 + W1 + 1, x_7: r6, x_8: H0^3*r7^3 - H0^3*r8^3 - 3*H0^2*r7^2 - (H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3)*r1^2 - r1^3 - (H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3*r1 + 3)*r2^2 + r2^3 + 3*(H0^3*r7 - H0^2)*r8^2 - (3*H0^2*r7^2 + 3*H0^2*r8^2 + H0*W1*r10 - 6*H0*r7 - 6*(H0^2*r7 - H0)*r8 - (2*H0^2*W1*r7 - 2*H0^2*W1*r8 - 2*H0*W1 + H0*W2)*r9 + H1 + W1 + 3)*r1 + (H0^2*W1*r7 - H0^2*W1*r8 - H0*W1 + H0*W2)*r10 + (3*H0^2*r7^2 + 3*H0^2*r8^2 + H0*W1*r10 + 2*(H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3)*r1 + 3*r1^2 - 6*H0*r7 - 6*(H0^2*r7 - H0)*r8 - (2*H0^2*W1*r7 - 2*H0^2*W1*r8 - 2*H0*W1 + H0*W2)*r9 + H1 + W1 + 3)*r2 + W3*r3 + H1*r6 + (H0*H1 + H0*W1 + 3*H0)*r7 - (3*H0^3*r7^2 - 6*H0^2*r7 + H

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 13 by 27 dense matrices over Symbolic Ring' and 'Vector space of dimension 31 over Symbolic Ring'

In [22]:
# Solving for TA using analytic methods

#print(Tw1)
#print(m_tens_h)

def solve_for_ta_analytic(chosen_sol):
    print("\n\n\n=================================")
    print(chosen_sol)
    
    # Tw1 = Tw1.subs(chosen_sol)
    # print(" Tw1:")
    # print(Tw1)
    # Tw2 = Tw2.subs(chosen_sol)
    # print(" Tw2:")
    # print(Tw2)
    
    # Tx1 = Tx1.subs(chosen_sol)
    # print(" Tx1:")
    # print(Tx1)
    # Tx2 = Tx2.subs(chosen_sol)
    # print(" Tx2:")
    # print(Tx2)
    # print("\n~~~~~~~~~~~~~~~~~")

    mts = []
    rts = []

    for t in range(nx):
        m_t = Matrix([[m_tens[i][j][t] for j in range(m)] for i in range(n)])
        r_t = Matrix([[sum(m_tens[i][j][l] * Tx1[l][t] * Tw1[j][k] for j in range(m) for l in range(nx)) for k in range(m)] for i in range(n)])
        #print("m_t rank")
        #print(m_t.rank())
        #print("r_t rank")
        #print(r_t.rank())
        mts.append(m_t)
        rts.append(r_t)
        print("M/R matrices, iter ", t)
        print("LHS:")
        print(m_t)
        print("====")
        print("RHS:")
        print(r_t)

        print("========\n")

    s = Matrix([[sum((m_tens_const[i][j] + sum(m_tens[i][j][l] * Tx2[l] for l in range(nx))) * Tw1[j][k] \
                     for j in range(m)) for k in range(m)] for i in range(n)])

    mhts = []
    rhts = []
    
    for l in range(nh):
        mh_l = Matrix([[m_tens_h[i][j][l] for j in range(m)] for i in range(n)])
        mhts.append(mh_l)
        rh_l = Matrix([[sum((m_tens_h[i][j][l]\
                           + sum(m_tens[i][j][s] * Tx3[s][l] for s in range(nx))) * Tw1[j][k] 
                            for j in range(m))\
                        for k in range(m)] for i in range(n)])
        rhts.append(rh_l)

        print("H matrices, iter ", l)
        print("LHS:")
        print(mh_l)
        print("====")
        print("RHS:")
        print(rh_l)
        print("RHS component 1:")
        print(Matrix([[sum((m_tens_h[i][j][l]) * Tw1[j][k] 
                            for j in range(m))\
                        for k in range(m)] for i in range(n)]))
        print("RHS component 2:")
        print(Matrix([[sum((sum(m_tens[i][j][s] * Tx3[s][l] for s in range(nx))) * Tw1[j][k] 
                            for j in range(m))\
                        for k in range(m)] for i in range(n)]))
        

        try:
            Tam1_l_alt = mh_l.solve_left(rh_l)
            print("! Solvable !")
        except ValueError:  
            print("!!!!!!!!!! NO SOLUTIONS FOUND")

        print("========\n")
        

    print("Matrix length params: ")
    print(nx*m)
    print(nh*m)
    print(1*m)
    
    #print("s rank")
    #print(s.rank())

    #print("m const rank")
    #print(Matrix(m_tens_const).rank())

    joint_matrix_lhs = block_matrix([mts + mhts + [Matrix(m_tens_const)]])
    joint_matrix_rhs = block_matrix([rts + rhts + [s]])

    #print("rank of joint matrices")
    #print(joint_matrix_lhs)
    #print(joint_matrix_lhs.rank())
    #print(joint_matrix_rhs)
    #print(joint_matrix_rhs.rank())

    print("joint matrix lhs rank: ", joint_matrix_lhs.rank())
    print("joint matrix lhs dimensions: ", joint_matrix_lhs.dimensions())
    print("joint matrix rhs dimensions: ", joint_matrix_rhs.dimensions())

    vlim = 13
    hlim = 130
    lhs_hlim = joint_matrix_lhs[:vlim,:hlim]
    rhs_hlim = joint_matrix_rhs[:vlim,:hlim]
    
    print(lhs_hlim)
    print("\n\n==================\n")
    print(rhs_hlim)

    print("\n\n==================\n")
    print(lhs_hlim.solve_left(rhs_hlim))
    
    #print("solution via solve_left:")
    # B.solve_left(C) is "find me A such that A * B = C"
    Tam1_alt = joint_matrix_lhs.solve_left(joint_matrix_rhs)
    #print(Tam_alt)

    Tam2_alt = Matrix([[sum(m_tens[i][s][l] * Tx1[l][j] * Tw2[s] for s in range(m) for l in range(nx)) for j in range(nx)] for i in range(n)])
    Tam_alt = block_matrix([[Tam1_alt,Tam2_alt]])
    print("Tam_alt:")
    print(Tam_alt)

    # Taa_alt = [sum(Tw2[j] * (m_tens_const[i][j] + sum(m_tens[i][j][l] * Tx2[l] for l in range(n))) for j in range(m)) for i in range(n)]


    # print("Taa_alt")
    # print(Taa_alt)
    
    # Tah_alt = [[sum(m_tens_h[l][k][j] + Tx3[i][j] * sum(m_tens[i][j][s] for s in range(nx)) for l in range(n)) for j in range(nh)] for i in range(n)]


    # print("Tah_alt")
    # print(Tah_alt)
    
for sol in sols:
    solve_for_ta_analytic(sol)




{x_0: r1, x_1: r2, x_2: r3, x_3: r4, x_4: -H0*W1*r9 + W1*r3 + H1*r4 + H0*r7 - H0*r8 - r1 + r2 - 1, x_5: r5, x_6: H0^2*r7^2 + H0^2*r8^2 + H0*W1*r10 + (H0*W1*r9 - 2*H0*r7 + 2*H0*r8 + 2)*r1 + r1^2 - (H0*W1*r9 - 2*H0*r7 + 2*H0*r8 + 2*r1 + 2)*r2 + r2^2 + W2*r3 + H1*r5 - 2*H0*r7 - 2*(H0^2*r7 - H0)*r8 - (H0^2*W1*r7 - H0^2*W1*r8 - H0*W1 + H0*W2)*r9 + H1 + W1 + 1, x_7: r6, x_8: H0^3*r7^3 - H0^3*r8^3 - 3*H0^2*r7^2 - (H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3)*r1^2 - r1^3 - (H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3*r1 + 3)*r2^2 + r2^3 + 3*(H0^3*r7 - H0^2)*r8^2 - (3*H0^2*r7^2 + 3*H0^2*r8^2 + H0*W1*r10 - 6*H0*r7 - 6*(H0^2*r7 - H0)*r8 - (2*H0^2*W1*r7 - 2*H0^2*W1*r8 - 2*H0*W1 + H0*W2)*r9 + H1 + W1 + 3)*r1 + (H0^2*W1*r7 - H0^2*W1*r8 - H0*W1 + H0*W2)*r10 + (3*H0^2*r7^2 + 3*H0^2*r8^2 + H0*W1*r10 + 2*(H0*W1*r9 - 3*H0*r7 + 3*H0*r8 + 3)*r1 + 3*r1^2 - 6*H0*r7 - 6*(H0^2*r7 - H0)*r8 - (2*H0^2*W1*r7 - 2*H0^2*W1*r8 - 2*H0*W1 + H0*W2)*r9 + H1 + W1 + 3)*r2 + W3*r3 + H1*r6 + (H0*H1 + H0*W1 + 3*H0)*r7 - (3*H0^3*r7^2 - 6*H0^2*r7 

ValueError: matrix equation has no solutions